In [9]:
import pandas as pd
df_ana = pd.read_csv("../dados/df_preparado.csv")

In [10]:
df_ana20 = df_ana.head(200)

In [13]:
# extractor.py
import os, json, re, ast
import openai
import pandas as pd
from dotenv import load_dotenv

load_dotenv()                      
openai.api_key = os.getenv("OPENAI_API_KEY")

# ---------- 1. prompt fixo ----------
_PROMPT_TEMPLATE = """
Você é um assistente de análise jurídica especialista em documentos CADE. Extraia as informações
solicitadas do documento a seguir.

Instruções específicas:
1. Para 'decisao_tribunal_IA': considere apenas os termos finais
   ('voto pelo arquivamento', 'voto pela condenação').
2. Para 'seguiu_nota_tecnica_IA': verifique se o relator menciona
   explicitamente seguir ou divergir da nota técnica.
3. Para valores de multa: extraia apenas valores explícitos
   no dispositivo final.
4. Para 'percentual_faturamento_IA': extraia o percentual da multa em relação ao faturamento da empresa
5. Quando não encontrar informação clara, use null.

Documento:
\"\"\"{texto}\"\"\"

Devolva APENAS JSON com os campos:
{{
  "decisao_tribunal_IA": "Condenação" | "Arquivamento" | null,
  "seguiu_nota_tecnica_IA": true | false | null,
  "tipo_infracao_concorrencial_IA": string | null,
  "multa_IA": true | false,
  "tipo_de_multa_IA": "valor_fixo" | "percentual" | "ambos" | null,
  "valor_multa_reais_IA": float | null,
  "percentual_faturamento_IA": float | null
}}
"""

def _safe_load_json(s: str) -> dict:
    """
    Tenta carregar primeiro com json.loads.
    Se falhar, usa ast.literal_eval como fallback para
    pequenos erros de formatação (aspas simples, True/False, etc.).
    """
    try:
        return json.loads(s)
    except json.JSONDecodeError:
        try:
            # Remove crases / backticks ocasionais
            cleaned = re.sub(r"[`´]", "", s)
            return ast.literal_eval(cleaned)
        except Exception as e:
            raise ValueError(f"Falha ao decodificar JSON: {e}") from None

def extrair_campos(texto: str,
                   model: str = "gpt-4o-mini",
                   temperature: float = 0.0) -> dict:
    prompt = _PROMPT_TEMPLATE.format(texto=texto)

    response = openai.chat.completions.create(
        model=model,
        temperature=temperature,
        messages=[{"role": "user", "content": prompt}],
        # força o retorno em JSON puro
        response_format={"type": "json_object"}
    )

    content_json = response.choices[0].message.content.strip()

    # agora quase sempre é JSON limpo; _safe_load_json é só por precaução
    return _safe_load_json(content_json)

# ---------- 3. aplicar no DataFrame ----------
def processar_dataframe(df: pd.DataFrame,
                        coluna_texto: str = "conteudo") -> pd.DataFrame:
    """
    Aplica extrair_campos a cada linha da coluna `coluna_texto`
    e devolve um novo DataFrame com as colunas expandidas.
    """
    resultados = df[coluna_texto].apply(extrair_campos)
    df_exp = pd.concat([df, resultados.apply(pd.Series)], axis=1)

    # garante tipos convenientes
    float_cols = ["valor_multa_reais_IA", "percentual_faturamento_IA"]
    for c in float_cols:
        df_exp[c] = pd.to_numeric(df_exp[c], errors="coerce")

    bool_cols = ["seguiu_nota_tecnica_IA", "multa_IA"]
    for c in bool_cols:
        df_exp[c] = df_exp[c].astype("boolean")

    return df_exp

    

In [ ]:
df_legal = processar_dataframe(df_ana20, coluna_texto="conteudo")
df_legal.to_csv("decisoes_enriquecidas.csv", index=False)

In [ ]:
df_legal.head(10)

,id,id_unique,numero_sei,ano_documento,assinaturas,descricao_tipo_documento,descricao_tipo_processo,numero_processo,id_unidade,sigla_unidade,...,decisao_tribunal,conteudo,diferenca_dias,decisao_tribunal,seguiu_nota_tecnica,tipo_infracao_concorrencial,multa,tipo_de_multa,valor_multa_reais,percentual_faturamento
0,abbdc450-7ff1-4990-9bcc-8ae4386592f0,jurisprudencia__1390607,1390607,2024,Outras,Voto Processo Administrativo,Processo Administrativo,08700.004558/2019-05,110000967.0,GAB1,...,vazio,SEI/CADE - 1390607 - Voto Processo Administrat...,1714,Condenação,True,cartel,True,valor_fixo,3.241162e+05,1.8
1,b41e8f1f-449e-4d28-a574-18f365d32b6f,jurisprudencia__1384594,1384594,2024,Gustavo Augusto Freitas de Lima,Voto Processo Administrativo,Processo Administrativo,08700.004974/2015-71,110000969.0,GAB3,...,condenacao,SEI/CADE - 1384594 - Voto Processo Administrat...,3276,Condenação,True,imposição de tabela de preços,True,valor_fixo,1.000000e+05,NaN
2,adec625d-4f5e-4514-b938-a295fdc1e8a4,jurisprudencia__1384685,1384685,2024,Outras,Voto,Processo Administrativo,08700.005915/2022-40,110001021.0,DIAP,...,arquivamento,SEI/CADE - 1384685 - Voto\n\n\nProcesso nº 087...,640,Arquivamento,True,None,False,None,NaN,NaN
3,c8499708-5778-45d9-8914-9ab97b77c1fd,jurisprudencia__1377990,1377990,2024,Alexandre Cordeiro Macedo,Voto Processo Administrativo,Processo Administrativo,08700.007776/2016-41,110000955.0,GAB-PRES,...,condenacao,SEI/CADE - 1377990 - Voto Processo Administrat...,2709,Arquivamento,True,Cartel em licitação,True,ambos,1.484524e+01,15.0
4,29758e3d-7047-4c5a-8ac6-b6815c67ec3d,jurisprudencia__1377023,1377023,2024,Gustavo Augusto Freitas de Lima,Voto Processo Administrativo,Processo Administrativo,08700.007776/2016-41,110000969.0,GAB3,...,condenacao,SEI/CADE - 1377023 - Voto Processo Administrat...,2709,Arquivamento,True,formação de cartel,True,percentual,NaN,12.0
5,15187e9b-d1e4-4c9d-97e2-721755174c1c,jurisprudencia__1377847,1377847,2024,Outras,Voto Processo Administrativo,Processo Administrativo,08700.007776/2016-41,110000972.0,GAB6,...,condenacao,SEI/CADE - 1377847 - Voto Processo Administrat...,2709,None,<NA>,None,False,None,NaN,NaN
6,2aabf3bb-af87-47ed-b09b-0726d04cdf5f,jurisprudencia__1377767,1377767,2024,Outras,Voto Processo Administrativo,Processo Administrativo,08700.007776/2016-41,110000967.0,GAB1,...,condenacao,SEI/CADE - 1377767 - Voto Processo Administrat...,2709,Condenação,True,None,True,valor_fixo,1.484524e+07,15.0
7,0d278d0f-5ab8-44cc-9ec6-cde5c399d9f9,jurisprudencia__1375782,1375782,2024,Gustavo Augusto Freitas de Lima,Voto Processo Administrativo,Processo Administrativo,08700.003266/2022-42,110000969.0,GAB3,...,arquivamento,SEI/CADE - 1375782 - Voto Processo Administrat...,706,Arquivamento,True,FPR (fixação de preço de revenda),False,None,NaN,NaN
8,84fe3a4b-a1b4-4536-957e-2aa5c372527f,jurisprudencia__1362271,1362271,2024,Gustavo Augusto Freitas de Lima,Voto Processo Administrativo,Processo Administrativo,08700.003699/2017-31,110000969.0,GAB3,...,vazio,SEI/CADE - 1362271 - Voto Processo Administrat...,2464,Condenação,False,cartel de licitação,True,valor_fixo,8.651777e+07,NaN
9,5c7e5985-7239-4ab1-9662-ddc610b2c48d,jurisprudencia__1358814,1358814,2024,Gustavo Augusto Freitas de Lima,Voto Processo Administrativo,Processo Administrativo,08012.006043/2008-37,110000969.0,GAB3,...,condenacao,SEI/CADE - 1358814 - Voto Processo Administrat...,5763,Arquivamento,<NA>,cartel,True,percentual,NaN,15.0
